# Laboratorio 6 

Francis Aguilar  
Gerardo Pineda  
Angela García  

# Task 1  

**1. En un juego de suma cero para dos jugadores, ¿cómo funciona el algoritmo minimax para determinar la estrategia óptima para cada jugador? ¿Puede explicarnos el concepto de "valor minimax" y su importancia en este contexto?**
El funcionamiento del algoritmo consiste en construir un árbol de juego donde cada nodo representa un estado del juego y las ramas corresponden a las posibles jugadas. En los niveles donde le toca jugar al primer jugador (Max), se elige la jugada con el mayor valor posible, mientras que en los niveles donde le toca jugar al oponente (Min), se elige la jugada con el menor valor. Este proceso se repite de manera recursiva hasta llegar al nodo raíz, lo que permite determinar la mejor jugada inicial.

El valor minimax de un nodo es el puntaje que un jugador puede asegurar si ambos juegan de manera óptima a partir de ese estado. Su importancia radica en que permite evaluar qué tan ventajosa o desventajosa es una posición en el juego, guiando a los jugadores hacia decisiones óptimas basadas en la anticipación de los movimientos del oponente.



**2. Compare y contraste el algoritmo minimax con la poda alfa-beta. ¿Cómo mejora la poda alfa-beta la eficiencia del algoritmo minimax, particularmente en árboles de caza grandes? Proporcione un ejemplo para ilustrar la diferencia en la complejidad computacional entre la poda minimax y alfa-beta.**
Para mejorar esta eficiencia, se utiliza la poda alfa-beta, que optimiza minimax descartando ramas del árbol que no influirán en la decisión final. Este algoritmo introduce dos valores, alfa y beta, que representan los mejores valores encontrados para los jugadores Max y Min, respectivamente. Si en algún momento un nodo tiene un valor que garantiza que el oponente nunca lo elegirá, se puede dejar de evaluar esa rama, reduciendo significativamente el número de nodos explorados.


**3. ¿Cuál es el papel de expectiminimax en juegos con incertidumbre, como aquellos que involucran nodos de azar o información oculta? ¿En qué se diferencia el expectiminimax del minimax en el manejo de resultados probabilísticos y cuáles son los desafíos clave que aborda?**

El algoritmo expectiminimax es una extensión de minimax diseñada para juegos en los que hay incertidumbre, como aquellos que incluyen azar o información oculta. Mientras que minimax asume que cada jugador tiene control total sobre sus decisiones, expectiminimax introduce nodos de azar, que representan eventos aleatorios como el lanzamiento de un dado o la repartición de cartas.

El funcionamiento del algoritmo es similar al minimax, con la diferencia de que, además de los nodos Max y Min, se incluyen nodos de azar. En estos nodos, en lugar de elegir el mejor o peor resultado, se calcula un valor esperado basado en las probabilidades de cada posible desenlace. Esto permite modelar situaciones en las que las decisiones no son completamente deterministas, sino que dependen de factores probabilísticos.

